# Pipeline
-Extract the gene counts for the whole genome in order to generate the minor allele frequencies for each of the variants
<br>
-Filter these variants by only those with MAFs < 0.001 (1e-03)
<br>
-Then generate the gene counts for the CMT diagnosed eid cohort, and merge with the filtered variants, leaving only CMT diagnosed variants with MAF < 0.0001

In [ ]:
%%bash 
# downloading plink
conda install bioconda::plink

In [ ]:
%%bash
dx download "Bulk/Exome sequences/Population level exome OQFE variants, PLINK format - final release/ukb23158_c*"

In [ ]:
%%bash
dx download "projects/CMT_NashBio/input_data/cmt_gene_location*.csv"
dx download "projects/CMT_NashBio/input_data/diagnosed_eid.txt"

## Generating minor allele frequencies (MAF) for each variant

In [ ]:
# script for MAF generation for each variant within CMT gene regions
# loading necessary packages
import pandas as pd
import subprocess

# Load gene location data
gene_location = pd.read_csv("cmt_gene_locations.csv")
gene_location_x = pd.read_csv("cmt_gene_location_x.csv")

# Chromosomes to process
ints = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]

# loop through chromosomes individually
for value in ints:
    # create temporary file for just genes within the chromosome being looped through
    temp = gene_location[gene_location["chrom"] == value]

    # creating the plink query to be run
    for _, gene in temp.iterrows():
        bfile = f"ukb23158_c{value}_b0_v1"
        start = int(gene["start"])
        end = int(gene["end"])
        gene_name = gene["gene"]
        out = f"{gene_name}_out"

        # Step 1: Extracting the region for each INDIVIDUAL GENE - first stage
        command1 = [
            "plink",
            "--bfile", bfile,
            "--chr", str(value),
            "--from-bp", str(start),
            "--to-bp", str(end),
            "--make-bed",
            "--out", out
        ]

        # Step 2: Calculate variant frequencies
        output2 = f"{gene_name}_freq_out"
        command2 = [
            "plink2",
            "--bfile", out,
            "--freq",
            "--max-maf", "0.001",
            "--out", output2
        ]

        try:
            subprocess.run(command1, check=True)
            subprocess.run(command2, check=True)
            print(f"Completed rare variant extraction for gene {gene_name} on chromosome {value}")
        except subprocess.CalledProcessError as e:
            print(f"Error processing gene {gene_name} on chromosome {value}: {e}")

# running same script but for chromosome X
for _, gene in gene_location_x.iterrows():
    bfile = f"ukb23158_cX_b0_v1"
    start = int(gene["start"])
    end = int(gene["end"])
    out = f"{gene['gene']}_out"

    command1 = [
        "plink",
        "--bfile", bfile,
        "--chr", "X",
        "--from-bp", str(start),
        "--to-bp", str(end),
        "--make-bed",  
        "--out", out
    ]

    output2 = f"{gene['gene']}_freq_out"
    command2 = [
        "plink",
        "--bfile", out,
        "--freq",
        "--max-maf", "0.0001",
        "--out", output2
    ]

    try:
        subprocess.run(command1, check=True)
        subprocess.run(command2, check=True)
        print(f"Completed rare variant extraction for gene {gene} on chromosome X")
    except subprocess.CalledProcessError as e:
        print(f"Error processing gene {gene} on chromosome X: {e}")

### Combining all .frq files into a single file
filter for MAF < 0.001

In [ ]:
import pandas as pd
import glob

# Function to read and clean .frq files
def read_freq_clean(file):
    # Assuming space-separated values with a header
    df = pd.read_csv(file, sep=' ', header=0)
    return df

# Get list of all .frq files in the current directory
freq_file_list = glob.glob("*.frq")

# Read and combine all .frq files
freq_data_list = [read_freq_clean(file) for file in freq_file_list]
combined_freq_data = pd.concat(freq_data_list, ignore_index=True)

# Filter for MAF < 0.001
filtered_freq_data = combined_freq_data[combined_freq_data['MAF'] < 0.001]


## filtering whole exome sequencing data to only include those diagnosed with CMT
Then, extract minor allele calls from each gene region, listed in cmt_gene_locations.csv file

In [ ]:
# filtering whole exome data to only include 192 individuals diagnosed with CMT
# then extracting gene regions to identify variants
# loading necessary packages
import pandas as pd
import subprocess

# Load gene location data
gene_loc = pd.read_csv("cmt_gene_locations.csv")

# Chromosomes to process
chromosomes = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]

# read eid of those diagnosed with CMT
with open("diagnosed_eid.txt", "r") as f:
    eids_file = f.name

# loop through chromosomes individually
for chrom in chromosomes:
    # generate temporary file for genes filtered by chromosome being looped through
    temp = gene_loc[gene_loc["chrom"] == chrom]
    
    bfile = f"ukb23158_c{chrom}_b0_v1"
    output1 = f"chr{chrom}_filtered"

    # generating PLINK query for filtering eids
    command1 = [
        "plink",
        "--bfile", bfile,
        "--keep", eids_file,
        "--make-bed",
        "--out", output1
    ]

    # running first query
    try:
        subprocess.run(command1, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error filtering chromosome {chrom}: {e}")
        continue

    # loop through each gene in temporary file to identify variants carried in each gene regions for CMT carriers
    for _, gene in temp.iterrows():
        start = int(gene["start"])
        end = int(gene["end"])
        gene_name = gene["gene"]
        output2 = f"{gene_name}_gene"
        # setting out query
        command2 = [
            "plink",
            "--bfile", output1,
            "--chr", str(chrom),
            "--from-bp", str(start),
            "--to-bp", str(end),
            "--recode", "vcf",
            "--out", output2
        ]

        # running query
        try:
            subprocess.run(command2, check=True)
            print(f"Completed rare variant extraction for gene {gene_name} on chromosome {chrom}")
        except subprocess.CalledProcessError as e:
            print(f"Error processing gene {gene_name} on chromosome {chrom}: {e}")


# Load eid data
with open("diagnosed_eid.txt", "r") as f:
    eids_file = f.name

# Load gene data for those on the X chromosome
gene_loc = pd.read_csv("cmt_gene_location_x.csv")


bfile = f"ukb23158_cX_b0_v1"
output1 = f"chrX_filtered"

# set out query for X chromosome filtering for gene region
command1 = [
    "plink",
    "--bfile", bfile,
    "--keep", eids_file,
    "--make-bed",
    "--out", output1
]
subprocess.run(command1, check=True)
for _, gene in gene_loc.iterrows():


    start = int(gene["start"])
    end = int(gene["end"])
    gene_name = gene["gene"]
    output2 = f"{gene_name}_gene"
    # query for filtering gene regions on X chromosome
    command2 = [
        "plink",
        "--bfile", "chrX_filtered",
        "--chr", "X",
        "--from-bp", str(start),
        "--to-bp", str(end),
        "--recode", "vcf",
        "--out", output2
    ]

    try:
        
        subprocess.run(command2, check=True)
        print(f"Completed rare variant extraction for gene {gene} on chromosome X")
    except subprocess.CalledProcessError as e:
        print(f"Error processing gene {gene} on chromosome X: {e}")


### Combining all gene output VCF into a single VCF file


In [ ]:
# combining all gene vcf files into a single file for analysis
# loading necessary packages
import pandas as pd
import glob

def read_vcf(file):
    # Read the file line by line to find the header
    with open(file, 'r') as f:
        lines = f.readlines()
    
    # Find the line number where the header starts
    header_line = next(i for i, line in enumerate(lines) if line.startswith('#CHROM'))
    
    # Read the VCF from the header line onward
    df = pd.read_csv(file, sep='\t', skiprows=header_line, header=0)
    return df

# List all VCF files ending with 'gene.vcf'
vcf_file_list = glob.glob("*gene.vcf")

# Read and combine all VCF files
vcf_data_list = [read_vcf(file) for file in vcf_file_list]
gene_vcf_unfiltered = pd.concat(vcf_data_list, ignore_index=True)

# filtering vcf to only include variants with MAF < 0.001
gene_vcf = pd.merge(gene_vcf_unfiltered, filtered_freq_data, on='ID', how='inner')

### Extract carrier and count data for CMT carrier minor allele calls
This data can then be used to construct a list of variants carried by individuals diagnosed with CMT
<br>
These variants are then extracted from the whole WES cohort

In [ ]:
import pandas as pd

# extracting carrier and count data for each of the variants identified

# function for extraction of carriers per variant
def extract_carriers(dataset):
    # Convert to DataFrame if not already
    df = pd.DataFrame(dataset)

    # Melt the DataFrame: first 8 columns are identifiers, rest are sample genotypes
    id_vars = df.columns[:8].tolist()
    melted = df.melt(id_vars=id_vars, var_name='sample', value_name='genotype')

    # Determine zygosity
    def get_zygosity(gt):
        if gt in ["0/1", "1/0"]:
            return "het"
        elif gt == "1/1":
            return "hom"
        else:
            return None

    melted['zygosity'] = melted['genotype'].apply(get_zygosity)

    # Filter out rows without zygosity
    result = melted[melted['zygosity'].notna()].copy()

    # Extract carrier ID (first 7 characters of sample name)
    result['carrier'] = result['sample'].str[:7]

    # Select and rename columns
    result = result[[
        df.columns[0],  # chrom
        df.columns[1],  # pos
        df.columns[2],  # ID
        df.columns[3],  # ref
        df.columns[4],  # alt
        'zygosity',
        'carrier'
    ]]

    return result

# generating carrier data using function
gene_carriers = extract_carriers(gene_vcf)

# generating count data for each variant identified
# Step 1: Count zygosity per ID
counts = gene_carriers.groupby(['ID', 'zygosity']).size().reset_index(name='Freq')

counts_wide = counts.pivot(index='ID', columns='zygosity', values='Freq').reset_index()

# set missing values to NA
counts_wide = counts_wide.fillna(pd.NA)

# Step 4: Rename columns (optional, if needed)
counts_wide.columns.name = None  # remove pivot column name
counts_wide = counts_wide.rename(columns={'het': 'het', 'hom': 'hom'})

# Extract distinct carrier metadata (excluding zygosity and carrier)
carrier_info = gene_carriers.drop(columns=['zygosity', 'carrier']).drop_duplicates()
final_counts = pd.merge(counts_wide, carrier_info, on='ID', how='left')

### Generating text files with the minor allele calls for each chromosome

In [ ]:
# generating chromosome level individual text files
# each list the variants found within the chromosome, to be used in --extract command with PLINK

import pandas as pd

# Get unique chromosomes
unique_chroms = final_counts['chrom'].unique()

# Loop through each chromosome
for chromosome in unique_chroms:
    # Subset data for this chromosome
    chrom_data = final_counts[final_counts['chrom'] == chromosome]

    # Select and deduplicate the 'ID' column
    df = chrom_data[['ID']].drop_duplicates()

    # Write to file without headers and quotes, using ':' as separator
    df.to_csv(f"chr{chromosome}.txt", index=False, header=False, sep=':', quoting=3)


# Extracting variants from whole population
-Generate txt files for each chromosome listing all variants
<br>
-Use this to extract (--extract) the variants for the entire population

In [ ]:
import subprocess

ints = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]

for value in ints:
    bfile = f"ukb23158_c{value}_b0_v1"
    output = f"chr{value}_out"
    snps = f"chr{value}.txt"
    command = [
        "plink",
        "--bfile", bfile,
        "--extract", snps,
        "--recode", "vcf",
        "--out", output
    ]
    
    try:
        subprocess.run(command, check=True)
        print(f"Completed chromosome {value}")
    except subprocess.CalledProcessError as e:
        print(f"Error processing chromosome {value}: {e}")


bfile = f"ukb23158_cX_b0_v1"
output = f"chrX_out"
snps = f"chr23.txt"
command = [
    "plink",
    "--bfile", bfile,
    "--extract", snps,
    "--recode", "vcf",
    "--out", output
]

try:
    subprocess.run(command, check=True)
    print(f"Completed chromosome {value}")
except subprocess.CalledProcessError as e:
    print(f"Error processing chromosome {value}: {e}")

### Combining whole population variant call VCF into a single file

In [ ]:
# combining all gene vcf files into a single file for analysis
# loading necessary packages
import pandas as pd
import glob

# List all VCF files ending with 'gene.vcf'
whole_list = glob.glob("*gene.vcf")

# Read and combine all VCF files
whole_data = [read_vcf(file) for file in whole_list]
whole_vcf = pd.concat(whole_data, ignore_index=True)

whole_vcf.to_csv("whole.vcf", sep='\t', index=False)

### Code for loading Ensembl's variant effect predictor
Used to annotate identified variants

In [ ]:
%%bash

# Install sudo annd other in foreground due to installation order dependencies
wget https://www.sudo.ws/dist/sudo-1.9.15p5.tar.gz \
  && tar xf sudo-1.9.15p5.tar.gz \
  && cd sudo-1.9.15p5 \
  && ./configure && make && make install \
  && cd .. && rm -f sudo-1.9.15p5.tar.gz && rm -rf sudo-1.9.15p5

# Install build-essential package
sudo apt update
sudo apt install -y build-essential libmysqlclient-dev

# Install Perl modules using cpanminus
curl -L https://cpanmin.us | sudo perl - App::cpanminus
sudo cpanm Archive::Zip DBI DBD::mysql Bio::Root::Version Bio::DB::HTS

# Clone and setup VEP
git clone https://github.com/Ensembl/ensembl-vep.git
cd ensembl-vep
perl INSTALL.pl --AUTO acfp --ASSEMBLY GRCh38 --PLUGINS all --SPECIES homo_sapiens

# Download Homo sapiens cache data for VEP
curl -O https://ftp.ensembl.org/pub/release-115/variation/indexed_vep_cache/homo_sapiens_vep_115_GRCh38.tar.gz
tar xzf homo_sapiens_vep_115_GRCh38.tar.gz
rm homo_sapiens_vep_115_GRCh38.tar.gz
cd ..

In [ ]:
input_vcf = "whole.vcf"
output_vcf = "whole_input.vcf"

with open(input_vcf, "r") as infile, open(output_vcf, "w") as outfile:
    for line in infile:
        if line.startswith("#"):
            # Write header lines unchanged
            outfile.write(line)
        else:
            fields = line.strip().split("\t")
            if fields[0] == "23":
                fields[0] = "X"
            outfile.write("\t".join(fields) + "\n")

In [ ]:
%%bash
# Annotate the SCN9A vcf file
./ensembl-vep/vep -i ./whole_input.vcf --format vcf --fork 4 --cache --dir_cache ./ensembl-vep --species homo_sapiens --pick --everything --canonical --vcf --buffer_size 100 --force_overwrite -o whole_anno.vcf

sed 's/INFO/Allele|Consequence|IMPACT|SYMBOL|Gene|Feature_type|Feature|BIOTYPE|EXON|INTRON|HGVSc|HGVSp|cDNA_position|CDS_position|Protein_position|Amino_acids|Codons|Existing_variation|DISTANCE|STRAND|FLAGS|VARIANT_CLASS|SYMBOL_SOURCE|HGNC_ID|CANONICAL|MANE|MANE_SELECT|MANE_PLUS_CLINICAL|TSL|APPRIS|CCDS|ENSP|SWISSPROT|TREMBL|UNIPARC|UNIPROT_ISOFORM|GENE_PHENO|SIFT|PolyPhen|DOMAINS|miRNA|HGVS_OFFSET|AF|AFR_AF|AMR_AF|EAS_AF|EUR_AF|SAS_AF|gnomADe_AF|gnomADe_AFR_AF|gnomADe_AMR_AF|gnomADe_ASJ_AF|gnomADe_EAS_AF|gnomADe_FIN_AF|gnomADe_MID_AF|gnomADe_NFE_AF|gnomADe_REMAINING_AF|gnomADe_SAS_AF|gnomADg_AF|gnomADg_AFR_AF|gnomADg_AMI_AF|gnomADg_AMR_AF|gnomADg_ASJ_AF|gnomADg_EAS_AF|gnomADg_FIN_AF|gnomADg_MID_AF|gnomADg_NFE_AF|gnomADg_REMAINING_AF|gnomADg_SAS_AF|MAX_AF|MAX_AF_POPS|CLIN_SIG|SOMATIC|PHENO|PUBMED|MOTIF_NAME|MOTIF_POS|HIGH_INF_POS|MOTIF_SCORE_CHANGE|TRANSCRIPTION_FACTORS/g' whole_anno.vcf > whole_anno1.vcf
sed 's/|/\t/g' whole_anno1.vcf > whole_anno_final.vcf